In [41]:
import json

In [42]:
with open ('measurements.json','r') as file:
    data =json.load(file)

exploring the structure of data

In [43]:
import pandas as pd

In [44]:
df=pd.DataFrame(data)
print(df.head())

       type                                    id        date  \
0  ailments  b22d5853-9cbf-4c1e-8522-13e43ea1fbfa  2024-06-13   
1       bbt  2024d303-00f6-4ee5-8d7c-c6a2ec49a9f8  2020-05-08   
2       bbt  cfa52acf-3595-4c43-b8b2-239e0dacf83f  2020-05-03   
3       bbt  6094a92b-5add-412b-a065-3c0922e9ff58  2020-10-31   
4       bbt  23a3bbdf-5364-40bf-a291-2295ccfc1723  2020-06-08   

                                   value  
0               [{'option': 'cold_flu'}]  
1   {'celsius': 35.0, 'excluded': False}  
2   {'celsius': 36.0, 'excluded': False}  
3  {'celsius': 35.94, 'excluded': False}  
4  {'celsius': 35.33, 'excluded': False}  


In [45]:
print(df.shape)

(1318, 4)


In [46]:
print(df.info)

<bound method DataFrame.info of           type                                    id        date  \
0     ailments  b22d5853-9cbf-4c1e-8522-13e43ea1fbfa  2024-06-13   
1          bbt  2024d303-00f6-4ee5-8d7c-c6a2ec49a9f8  2020-05-08   
2          bbt  cfa52acf-3595-4c43-b8b2-239e0dacf83f  2020-05-03   
3          bbt  6094a92b-5add-412b-a065-3c0922e9ff58  2020-10-31   
4          bbt  23a3bbdf-5364-40bf-a291-2295ccfc1723  2020-06-08   
...        ...                                   ...         ...   
1313     stool  bf32f15b-466a-46f3-bb8a-6fdb487bda5c  2024-08-27   
1314     stool  0dc656e8-7aad-47e7-ae38-2226cba782e2  2024-08-28   
1315     stool  223a1f1a-6591-4933-9b27-b874669f74bd  2024-08-29   
1316      tags  a9aed0ed-a02e-4d9d-a8ca-83676433cf2c  2020-04-05   
1317    weight  44fd85d0-568d-4e3a-805f-ef23198e03ab  2024-04-02   

                                                  value  
0                              [{'option': 'cold_flu'}]  
1                  {'celsius': 35.0

how many unique values of id are there? Is it a primary key?

In [47]:
print(df['id'].nunique())

1318


unique values of date?

In [48]:
print(df['date'].nunique())

256


In [49]:
print(df['type'].nunique())

23


I dropped the column id because it will not provide any insight

In [50]:
df = df.drop(columns=['id'])

In [51]:
print(df.columns)

Index(['type', 'date', 'value'], dtype='object')


seeing what kind of data does "value" column has

In [52]:
for i, val in enumerate(df['value'].head(10), 1):
    print(f"{i}. {val}\n")


1. [{'option': 'cold_flu'}]

2. {'celsius': 35.0, 'excluded': False}

3. {'celsius': 36.0, 'excluded': False}

4. {'celsius': 35.94, 'excluded': False}

5. {'celsius': 35.33, 'excluded': False}

6. {'celsius': 35.56, 'excluded': False}

7. {'celsius': 34.4, 'excluded': False}

8. {'celsius': 35.33, 'excluded': False}

9. [{'option': 'pad'}]

10. [{'option': 'pad'}]



checking the type of each "value" column; whether it is dict or list or anything

In [53]:
df['value_type'] = df['value'].apply(type)
print(df['value_type'].value_counts())


value_type
<class 'list'>    974
<class 'dict'>    344
Name: count, dtype: int64


only seeing value  where "value" column has dictionary data type

In [54]:
df[df['value'].apply(lambda x: isinstance(x, dict))]['value'].head()


1     {'celsius': 35.0, 'excluded': False}
2     {'celsius': 36.0, 'excluded': False}
3    {'celsius': 35.94, 'excluded': False}
4    {'celsius': 35.33, 'excluded': False}
5    {'celsius': 35.56, 'excluded': False}
Name: value, dtype: object

only looking at the value where "value" column has list data type

In [55]:
df[df['value'].apply(lambda x: isinstance(x, list))]['value'].head()


0     [{'option': 'cold_flu'}]
8          [{'option': 'pad'}]
9          [{'option': 'pad'}]
10         [{'option': 'pad'}]
11         [{'option': 'pad'}]
Name: value, dtype: object

although it is shown as list, it is still dict. It is interperted as list because of [] being inside this structure.

the value describes the column "type". so I want to see them together to make sense.

In [56]:
print(df[['type', 'value']].head)

<bound method NDFrame.head of           type                                              value
0     ailments                           [{'option': 'cold_flu'}]
1          bbt               {'celsius': 35.0, 'excluded': False}
2          bbt               {'celsius': 36.0, 'excluded': False}
3          bbt              {'celsius': 35.94, 'excluded': False}
4          bbt              {'celsius': 35.33, 'excluded': False}
...        ...                                                ...
1313     stool                           [{'option': 'diarrhea'}]
1314     stool                                 [{'option': 'ok'}]
1315     stool                                 [{'option': 'ok'}]
1316      tags  [{'option': '90/60'}, {'option': 'blood pressu...
1317    weight                                {'kilograms': 52.0}

[1318 rows x 2 columns]>


### I have to convert different types into rows and place their respective values under them by pivoting around date.

But first I need to know what are the different kinds of values in "type" column

In [57]:
print(df['type'].unique())

['ailments' 'bbt' 'collection_method' 'craving' 'digestion' 'discharge'
 'energy' 'exercise' 'feelings' 'hair' 'leisure' 'mind' 'pain' 'period'
 'pms' 'sex_life' 'skin' 'sleep_duration' 'social_life' 'spotting' 'stool'
 'tags' 'weight']


In [58]:
print(df['type'].value_counts())


type
period               224
energy               134
collection_method    127
feelings             125
mind                 103
sleep_duration       101
social_life           90
pain                  84
discharge             82
exercise              65
stool                 60
spotting              29
skin                  27
digestion             21
hair                  20
pms                   11
bbt                    7
leisure                3
tags                   1
ailments               1
sex_life               1
craving                1
weight                 1
Name: count, dtype: int64


Let's see what type of "value" do type with count <10 have

In [59]:
df[df['type'].isin(['tags','ailments','sex_life','craving','weight','bbt','leisure'])]

,type,date,value,value_type
0,ailments,2024-06-13,[{'option': 'cold_flu'}],<class 'list'>
1,bbt,2020-05-08,"{'celsius': 35.0, 'excluded': False}",<class 'dict'>
2,bbt,2020-05-03,"{'celsius': 36.0, 'excluded': False}",<class 'dict'>
3,bbt,2020-10-31,"{'celsius': 35.94, 'excluded': False}",<class 'dict'>
4,bbt,2020-06-08,"{'celsius': 35.33, 'excluded': False}",<class 'dict'>
5,bbt,2020-05-10,"{'celsius': 35.56, 'excluded': False}",<class 'dict'>
6,bbt,2020-04-05,"{'celsius': 34.4, 'excluded': False}",<class 'dict'>
7,bbt,2020-05-09,"{'celsius': 35.33, 'excluded': False}",<class 'dict'>
135,craving,2024-01-31,[{'option': 'sweet'}],<class 'list'>
583,leisure,2024-01-31,[{'option': 'date'}],<class 'list'>


I am going to remove all these rows from bbt to weight because from there they have small count of that type to provide any insight

In [60]:
df=df[~df['type'].isin(['bbt','leisure','tags','ailments','sex_life','craving','weight'])]

In [61]:
print(df['type'].value_counts())

type
period               224
energy               134
collection_method    127
feelings             125
mind                 103
sleep_duration       101
social_life           90
pain                  84
discharge             82
exercise              65
stool                 60
spotting              29
skin                  27
digestion             21
hair                  20
pms                   11
Name: count, dtype: int64


out of 23 different types now 16 types remain

I still want to see how many list values remain and how many dict

In [62]:
df['value_type'] = df['value'].apply(type)
print(df['value_type'].value_counts())

value_type
<class 'list'>    967
<class 'dict'>    336
Name: count, dtype: int64


seeing what value each "type" column has

In [63]:
for t in df['type'].unique():
    print(f"\n--- TYPE: {t} ---")
    sample_values = df[df['type'] == t]['value']
    for v in sample_values:
        print(v)


--- TYPE: collection_method ---
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'option': 'pad'}]
[{'opti

from the above output, I saw that only one type{sleep_duration} has value in minutes like "minute": 450; otherwise all other have options.

I am going to remove "minutes: 450" in sleep_duration and only keep numeric values.

In [64]:
df.loc[df['type'] == 'sleep_duration', 'value'] = (
    df.loc[df['type'] == 'sleep_duration', 'value']
    .apply(lambda x: x['minutes'] if isinstance(x, dict) and 'minutes' in x else x)
)

Looking at the new numeric values in sleep_duration type.

In [65]:
df['value_type'] = df['value'].apply(type)
print(df[df['type'] == 'sleep_duration'][['value', 'value_type']])

     value     value_type
1036   540  <class 'int'>
1037   450  <class 'int'>
1038   450  <class 'int'>
1039   450  <class 'int'>
1040   450  <class 'int'>
...    ...            ...
1132   450  <class 'int'>
1133   450  <class 'int'>
1134   450  <class 'int'>
1135   450  <class 'int'>
1136   480  <class 'int'>

[101 rows x 2 columns]


Next thing I have noticed is that all dict datatypes in "value" column have only one option so I want to remove the key part and only keep value.

In [66]:
df['value'] = df['value'].apply(
    lambda x: x['option'] if isinstance(x, dict) and 'option' in x else x
)

what are the new data types of "value" columns?

In [67]:
df['value_type'] = df['value'].apply(type)
print(df['value_type'].value_counts())

value_type
<class 'list'>    967
<class 'str'>     235
<class 'int'>     101
Name: count, dtype: int64


collection method type only has 1 value which is pad. since it is a constant it will not add any value, so I removed it.

In [68]:
df = df[df['type'] != 'collection_method']

pms also only has "yes" as an option, so I remove it. and most of the value in stool have ok; so no insight from it.

In [69]:
df=df[~df['type'].isin(['stool','pms'])]

looking at the data again

In [70]:
for t in df['type'].unique():
    print(f"\n--- TYPE: {t} ---")
    sample_values = df[df['type'] == t]['value']
    for v in sample_values:
        print(v)


--- TYPE: digestion ---
[{'option': 'ok'}]
[{'option': 'gassy'}]
[{'option': 'bloated'}]
[{'option': 'ok'}]
[{'option': 'ok'}]
[{'option': 'gassy'}]
[{'option': 'bloated'}]
[{'option': 'ok'}]
[{'option': 'bloated'}, {'option': 'gassy'}]
[{'option': 'ok'}]
[{'option': 'bloated'}]
[{'option': 'bloated'}]
[{'option': 'ok'}]
[{'option': 'bloated'}]
[{'option': 'ok'}]
[{'option': 'ok'}]
[{'option': 'ok'}]
[{'option': 'ok'}]
[{'option': 'ok'}]
[{'option': 'gassy'}]
[{'option': 'gassy'}]

--- TYPE: discharge ---
[{'option': 'creamy'}]
[{'option': 'creamy'}]
[{'option': 'sticky'}]
[{'option': 'sticky'}]
[{'option': 'sticky'}]
[{'option': 'sticky'}]
[{'option': 'creamy'}]
[{'option': 'creamy'}]
[{'option': 'creamy'}]
[{'option': 'atypical'}]
[{'option': 'creamy'}]
[{'option': 'creamy'}]
[{'option': 'sticky'}]
[{'option': 'creamy'}]
[{'option': 'creamy'}]
[{'option': 'atypical'}]
[{'option': 'creamy'}]
[{'option': 'creamy'}]
[{'option': 'creamy'}]
[{'option': 'sticky'}]
[{'option': 'creamy'}]
[

Now for some values of types there are only one option in "value" column so I am going to extract them into string. So that when I make them columns they will have only one values.